In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm,linear_model,naive_bayes,tree,neighbors,ensemble,discriminant_analysis
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn import metrics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
#from pandas.tools.plotting import scatter_matrix

C:\Users\Saurav\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
X_train=pd.read_csv('train.csv')
X_t=X_train.copy()
X_test=pd.read_csv('test.csv')
X=pd.concat([X_train,X_test])
X_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [4]:
med=X[X['Pclass']==3]['Fare'].median()
X.loc[X['Ticket']=='3701',['Fare']]=med

In [5]:
X.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          2
dtype: int64

In [6]:
X[X['Embarked'].isnull()]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
61,62,1.0,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
829,830,1.0,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [7]:
X_train[['Embarked','Survived']].groupby(['Embarked'],as_index=False).mean().sort_values(by='Survived')

,Embarked,Survived
2,S,0.336957
1,Q,0.389610
0,C,0.553571


In [197]:
X.loc[X['Ticket']=='113572',['Embarked']]='S'

In [198]:
X.Age.isnull().sum()

263

In [199]:
X['FamilyName']=X['Name'].apply(lambda x: x.split(',')[0])
X['Name']=X['Name'].apply(lambda x: x.split(',')[1])
X['Salut']=X['Name'].apply(lambda x:x.split('.')[0])
X['Name']=X['Name'].apply(lambda x:x.split('.')[1])

In [200]:
X['Salut']=X['Salut'].apply(lambda x:x.strip())
#X[['Salut','Survived']].groupby(['Salut']).sum().sort_values(by='Survived')
X['Salut'].value_counts()

Mr              757
Miss            260
Mrs             197
Master           61
Dr                8
Rev               8
Col               4
Major             2
Mlle              2
Ms                2
Dona              1
the Countess      1
Lady              1
Sir               1
Mme               1
Jonkheer          1
Capt              1
Don               1
Name: Salut, dtype: int64

In [201]:
for i in range(len(X)):
    if((X.iloc[i,X.columns.get_loc('Sex')]=='male') and (X.iloc[i,X.columns.get_loc('Age')]<=14)):
        X.iloc[i,X.columns.get_loc('Salut')]=1
    if((X.iloc[i,X.columns.get_loc('Sex')]=='female') and (X.iloc[i,X.columns.get_loc('Age')]<=14)):
        X.iloc[i,X.columns.get_loc('Salut')]=2
    if((X.iloc[i,X.columns.get_loc('Sex')]=='male') and (X.iloc[i,X.columns.get_loc('Age')]>14)):
        X.iloc[i,X.columns.get_loc('Salut')]=3
    if((X.iloc[i,X.columns.get_loc('Sex')]=='female') and (X.iloc[i,X.columns.get_loc('Age')]>14)):
        X.iloc[i,X.columns.get_loc('Salut')]=4
    

In [202]:
Master=X[X['Salut']==1]['Age'].mean()
Miss=X[X['Salut']==2]['Age'].mean()
Mr=X[X['Salut']==3]['Age'].mean()
Mrs=X[X['Salut']==4]['Age'].mean()
Master
Miss

5.9118

In [203]:
for i in range(len(X)):
    if X.iloc[i,X.columns.get_loc('Salut')] in ['Dr','Capt','Col','Don','Dona',
    'Jonkheer','Lady','Major','Mlle','Mme','Mr','Mrs','Ms','Rev','Sir','the Countess']:
       X.iloc[i,X.columns.get_loc('Age')]=32
    if X.iloc[i,X.columns.get_loc('Salut')] in ['Master','Miss']:
        X.iloc[i,X.columns.get_loc('Age')]=8
    if X.iloc[i,X.columns.get_loc('Salut')] in ['Dr','Capt','Col','Don','Jonkheer','Major','Mr','Rev','Sir']:
        X.iloc[i,X.columns.get_loc('Salut')]=3
    if X.iloc[i,X.columns.get_loc('Salut')] in ['Dona','Lady','Miss','Mlle','Mme','Mrs','Ms']:
        X.iloc[i,X.columns.get_loc('Salut')]=4
    if X.iloc[i,X.columns.get_loc('Salut')] in ['Master']:
        X.iloc[i,X.columns.get_loc('Salut')]=1

In [204]:

X.drop(columns=['Name','FamilyName','Cabin','Ticket'],inplace=True)
X_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [205]:
cleanup_maps = {"Sex":     {"male": 1, "female": 2},
                "Embarked": {"S": 1, "Q": 2, "C": 3}}
X.replace(cleanup_maps, inplace=True)
X.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Salut
0,1,0.0,3,1,22.0,1,0,7.2500,1,3
1,2,1.0,1,2,38.0,1,0,71.2833,3,4
2,3,1.0,3,2,26.0,0,0,7.9250,1,4
3,4,1.0,1,2,35.0,1,0,53.1000,1,4
4,5,0.0,3,1,35.0,0,0,8.0500,1,3


In [206]:
X.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Salut
0,1,0.0,3,1,22.0,1,0,7.2500,1,3
1,2,1.0,1,2,38.0,1,0,71.2833,3,4
2,3,1.0,3,2,26.0,0,0,7.9250,1,4
3,4,1.0,1,2,35.0,1,0,53.1000,1,4
4,5,0.0,3,1,35.0,0,0,8.0500,1,3
5,6,0.0,3,1,32.0,0,0,8.4583,2,3
6,7,0.0,1,1,54.0,0,0,51.8625,1,3
7,8,0.0,3,1,2.0,3,1,21.0750,1,1
8,9,1.0,3,2,27.0,0,2,11.1333,1,4
9,10,1.0,2,2,14.0,1,0,30.0708,3,2


In [211]:
X_train=X.iloc[0:891,:]
X_test=X.iloc[891:,:]
X_test

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Salut
0,892,NaN,3,1,34.5,0,0,7.8292,2,3
1,893,NaN,3,2,47.0,1,0,7.0000,1,4
2,894,NaN,2,1,62.0,0,0,9.6875,2,3
3,895,NaN,3,1,27.0,0,0,8.6625,1,3
4,896,NaN,3,2,22.0,1,1,12.2875,1,4
...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,1,32.0,0,0,8.0500,1,3
414,1306,NaN,1,2,39.0,0,0,108.9000,3,4
415,1307,NaN,3,1,38.5,0,0,7.2500,1,3
416,1308,NaN,3,1,32.0,0,0,8.0500,1,3


In [212]:
train_passengerId=X_train['PassengerId']
test_passengerId=X_test['PassengerId']
y=X_train['Survived']
X_train.drop(columns=['PassengerId','Survived'],inplace=True)
X_test.drop(columns=['PassengerId','Survived'],inplace=True)

C:\Users\Saurav\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [213]:
X_train.shape

(891, 8)

In [215]:
X_test.shape

(418, 8)

In [216]:
dt=tree.DecisionTreeClassifier(max_depth=6)
dt.fit(X_train,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [217]:
pred=dt.predict(X_test)

In [223]:
out=pd.DataFrame({'PassengerId':test_passengerId , 'Survived':pred})
out['Survived']=out['Survived'].astype(int)
out.to_csv('submit.csv',index=False)

In [224]:
out.shape

(418, 2)